In [1]:
import pandas as pd
import helper as h
from google.cloud import bigquery
client = bigquery.Client()

from github3 import login
password = h.get_gh_credentials()
gh = login('bps10', password=password)

12


In [2]:
user = 'bps10'
user_profile = h.get_user_info(gh, user)

In [3]:
user_profile

{'login': 'bps10',
 'followers_count': 1,
 'following_count': 2,
 'bio': 'Neuroscientist, postdoc',
 'blog': 'https://bps10.github.io',
 'company': 'UC Berkeley @RoordaLab',
 'created_at': datetime.datetime(2012, 7, 12, 4, 14, 27, tzinfo=tzutc()),
 'public_repos_count': 28,
 'public_gists_count': 6,
 'hireable': True,
 'updated_at': datetime.datetime(2019, 1, 25, 15, 25, 3, tzinfo=tzutc())}

In [67]:
# Set use_legacy_sql to True to use legacy SQL syntax.
job_config = bigquery.QueryJobConfig()
job_config.use_legacy_sql = True

query = (
"""SELECT  
  COUNT(type) as event_count, 
  MAX(created_at) as last_event, 
  MIN(created_at) as first_event,
  SUM(CASE WHEN type = 'CommitCommentEvent' then 1 else 0 end) as CommitCommentEvent_count,
  SUM(CASE WHEN type = 'CreateEvent' then 1 else 0 end) as CreateEvent_count,
  SUM(CASE WHEN type = 'DeleteEvent' then 1 else 0 end) as DeleteEvent_count,
  SUM(CASE WHEN type = 'ForkEvent' then 1 else 0 end) as ForkEvent_count,
  SUM(CASE WHEN type = 'GollumEvent' then 1 else 0 end) as GollumEvent_count,
  SUM(CASE WHEN type = 'IssueCommentEvent' then 1 else 0 end) as IssueCommentEvent_count,
  SUM(CASE WHEN type = 'IssuesEvent' then 1 else 0 end) as IssuesEvent_count,
  SUM(CASE WHEN type = 'MemberEvent' then 1 else 0 end) as MemberEvent_count,
  SUM(CASE WHEN type = 'PublicEvent' then 1 else 0 end) as PublicEvent_count,
  SUM(CASE WHEN type = 'PullRequestEvent' then 1 else 0 end) as PullRequestEvent_count,
  SUM(CASE WHEN type = 'PullRequestReviewCommentEvent' then 1 else 0 end) as PullRequestReviewCommentEvent_count,
  SUM(CASE WHEN type = 'PushEvent' then 1 else 0 end) as PushEvent_count,
  SUM(CASE WHEN type = 'ReleaseEvent' then 1 else 0 end) as ReleaseEvent_count,
  SUM(CASE WHEN type = 'WatchEvent' then 1 else 0 end) as WatchEvent_count,

  FROM 
  [githubarchive:year.2018]
WHERE (actor.login = '""" + user + """' AND public);

"""
    
)

#results_df = gbq.read_gbq(query_job, project_id=project_id, private_key='path_to_my.json')

df = client.query(query, location="US", job_config=job_config).to_dataframe()

In [68]:
df.head()

,event_count,last_event,first_event,CommitCommentEvent_count,CreateEvent_count,DeleteEvent_count,ForkEvent_count,GollumEvent_count,IssueCommentEvent_count,IssuesEvent_count,MemberEvent_count,PublicEvent_count,PullRequestEvent_count,PullRequestReviewCommentEvent_count,PushEvent_count,ReleaseEvent_count,WatchEvent_count
0,163,2019-01-04 21:58:39+00:00,2018-02-06 21:26:44+00:00,0,13,0,3,9,0,0,2,1,25,0,109,0,1


In [49]:
user_profile = pd.DataFrame(user_profile, index=[0])
user_profile.head()

,login,followers_count,following_count,bio,blog,company,created_at,public_repos_count,public_gists_count,hireable,updated_at
0,bps10,1,2,"Neuroscientist, postdoc",https://bps10.github.io,UC Berkeley @RoordaLab,2012-07-12 04:14:27+00:00,28,6,True,2018-11-24 17:02:47+00:00


In [50]:
user = user_profile.join(df)
user.transpose()

,0
login,bps10
followers_count,1
following_count,2
bio,"Neuroscientist, postdoc"
blog,https://bps10.github.io
company,UC Berkeley @RoordaLab
created_at,2012-07-12 04:14:27+00:00
public_repos_count,28
public_gists_count,6
hireable,True


In [51]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('test').getOrCreate()

In [52]:
from pyspark.sql.types import IntegerType
for col, t in spark_user.dtypes:
    if t == 'bigint':
        spark_user = spark_user.withColumn(col,
                                           spark_user[col].cast(IntegerType()))

In [53]:
spark_user = spark.createDataFrame(user)
spark_user = spark_user.drop('bio')

In [54]:
spark_user

DataFrame[login: string, followers_count: bigint, following_count: bigint, blog: string, company: string, created_at: timestamp, public_repos_count: bigint, public_gists_count: bigint, hireable: boolean, updated_at: timestamp, event_count: bigint, last_event: timestamp, first_event: timestamp, CommitCommentEvent_count: bigint, CreateEvent_count: bigint, DeleteEvent_count: bigint, ForkEvent_count: bigint, GollumEvent_count: bigint, IssueCommentEvent_count: bigint, IssuesEvent_count: bigint, MemberEvent_count: bigint, PublicEvent_count: bigint, PullRequestEvent_count: bigint, PullRequestReviewCommentEvent_count: bigint, PushEvent_count: bigint, ReleaseEvent_count: bigint, WatchEvent_count: bigint]

In [55]:
spark_user.printSchema()

root
 |-- login: string (nullable = true)
 |-- followers_count: long (nullable = true)
 |-- following_count: long (nullable = true)
 |-- blog: string (nullable = true)
 |-- company: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- public_repos_count: long (nullable = true)
 |-- public_gists_count: long (nullable = true)
 |-- hireable: boolean (nullable = true)
 |-- updated_at: timestamp (nullable = true)
 |-- event_count: long (nullable = true)
 |-- last_event: timestamp (nullable = true)
 |-- first_event: timestamp (nullable = true)
 |-- CommitCommentEvent_count: long (nullable = true)
 |-- CreateEvent_count: long (nullable = true)
 |-- DeleteEvent_count: long (nullable = true)
 |-- ForkEvent_count: long (nullable = true)
 |-- GollumEvent_count: long (nullable = true)
 |-- IssueCommentEvent_count: long (nullable = true)
 |-- IssuesEvent_count: long (nullable = true)
 |-- MemberEvent_count: long (nullable = true)
 |-- PublicEvent_count: long (nullable = true)


### Fetching user event history data

In [83]:
job_config = bigquery.QueryJobConfig()
job_config.use_legacy_sql = True
query = (
"""SELECT  
  COUNT(type) as event_count, 
  MAX(created_at) as last_event, 
  MIN(created_at) as first_event,
  SUM(CASE WHEN type = 'CommitCommentEvent' then 1 else 0 end) as CommitCommentEvent_count,
  SUM(CASE WHEN type = 'CreateEvent' then 1 else 0 end) as CreateEvent_count,
  SUM(CASE WHEN type = 'DeleteEvent' then 1 else 0 end) as DeleteEvent_count,
  SUM(CASE WHEN type = 'ForkEvent' then 1 else 0 end) as ForkEvent_count,
  SUM(CASE WHEN type = 'GollumEvent' then 1 else 0 end) as GollumEvent_count,
  SUM(CASE WHEN type = 'IssueCommentEvent' then 1 else 0 end) as IssueCommentEvent_count,
  SUM(CASE WHEN type = 'IssuesEvent' then 1 else 0 end) as IssuesEvent_count,
  SUM(CASE WHEN type = 'MemberEvent' then 1 else 0 end) as MemberEvent_count,
  SUM(CASE WHEN type = 'PublicEvent' then 1 else 0 end) as PublicEvent_count,
  SUM(CASE WHEN type = 'PullRequestEvent' then 1 else 0 end) as PullRequestEvent_count,
  SUM(CASE WHEN type = 'PullRequestReviewCommentEvent' then 1 else 0 end) as PullRequestReviewCommentEvent_count,
  SUM(CASE WHEN type = 'PushEvent' then 1 else 0 end) as PushEvent_count,
  SUM(CASE WHEN type = 'ReleaseEvent' then 1 else 0 end) as ReleaseEvent_count,
  SUM(CASE WHEN type = 'WatchEvent' then 1 else 0 end) as WatchEvent_count,

  FROM 
  [githubarchive:year.2015],
  [githubarchive:year.2016],
  [githubarchive:year.2017],
  [githubarchive:year.2018]
WHERE (actor.login = '""" + user + """' AND public = true);

"""
    
)

#results_df = gbq.read_gbq(query_job, project_id=project_id, private_key='path_to_my.json')

df = client.query(query, location="US", job_config=job_config).to_dataframe()

In [84]:
df

,event_count,last_event,first_event,CommitCommentEvent_count,CreateEvent_count,DeleteEvent_count,ForkEvent_count,GollumEvent_count,IssueCommentEvent_count,IssuesEvent_count,MemberEvent_count,PublicEvent_count,PullRequestEvent_count,PullRequestReviewCommentEvent_count,PushEvent_count,ReleaseEvent_count,WatchEvent_count
0,187,2019-01-04 21:58:39+00:00,2015-01-08 18:04:14+00:00,0,14,0,3,9,0,0,2,1,25,0,132,0,1
